In [11]:
import pandas as pd
import numpy as np
import os, sys
import random
import pickle

import matplotlib.pyplot as plt
import copy
from collections import Counter
import warnings

sys.path.insert(0, '/nas/longleaf/home/smerrill/evolution/')
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 10)
warnings.filterwarnings("ignore")

# This Notebook Is for Analyzing Fitness Functions
- It first requires running cheating_analysis.py

### 1. Load and process results
- NOTE: These functions here may require some subtle tweaks based on how you save the files so take caution here

In [12]:
def add_difference_cols(results_df2, cols_to_diff):
    results_df = copy.deepcopy(results_df2)
    start_cols = results_df.columns
    for col in cols_to_diff:
        new_cols = [x for x in start_cols if col in x]
        replace_string =  '_' + col.split('_')[-1]
        orig_cols = [x.replace(replace_string, '') for x in copy.deepcopy(new_cols)]
        
        for i in range(len(orig_cols)-1, -1, -1):
            if orig_cols[i] not in results_df.columns:
                print(orig_cols[i])
                del new_cols[i]
                del orig_cols[i]
        
        diff_cols = [x + '_diff' for x in copy.deepcopy(new_cols)]    
        #print(results_df[new_cols])
        results_df[diff_cols] = (results_df[orig_cols].values - results_df[new_cols].values)
        results_df[diff_cols] = results_df[diff_cols].values/results_df[orig_cols].values
        results_df[diff_cols] = results_df[diff_cols].fillna(0)
    return results_df

def get_results_df(save_path):
    exps = os.listdir(save_path)
    exps.sort()
    
    # concatonte all resutls_df
    results_df = pd.DataFrame()
    for exp in exps:
        try:
            tmp = pd.read_csv(save_path + f'/{exp}/results_df.csv')
            if tmp.empty:
                continue
            results_df = pd.concat([results_df, tmp])
        except Exception as e:
            print(e)
    print(results_df.shape)
    results_df = results_df.fillna(0)
    results_df = results_df.loc[:, (results_df.sum(axis=0) != 0)]
    
    for col in results_df.columns:
        try:
            results_df[col] = results_df[col].astype(float)
        except ValueError:
            results_df.drop(columns=[col], inplace=True)

    cols_to_diff = ['noise', # these will actually update all columns at begining of loop
                'corrupt',# but these will be corrected as we iterate over list
                'val_id_noise', 
               'val_id_corrupt', ]
               #'val_ood_noise',
               #'val_ood_corrupt']

    results_df = add_difference_cols(results_df, cols_to_diff)
    results_df = results_df.reset_index(drop=True)
    return results_df

In [4]:
save_path = '/proj/mcavoy_lab/data/evo_models/exps/cheating_fitness_0802_EOD/german'
results_df = get_results_df(save_path)

(14516, 1285)
min_std_mmd_val_id
min_mmd_val_id
min_std_mmd_val_id
min_mmd_val_id


In [4]:
results_df

Feature_0  Feature_1  Feature_2  Feature_3  Feature_4  Feature_5  \
0       0.527197   0.464435   0.460251   0.493724   0.476987   0.481172   
1       0.522727   0.477273   0.500000   0.522727   0.477273   0.454545   
2       0.462312   0.512563   0.507538   0.512563   0.482412   0.487437   
3       0.584158   0.495050   0.514851   0.485149   0.514851   0.445545   
4       0.441441   0.459459   0.405405   0.531532   0.486486   0.468468   
...          ...        ...        ...        ...        ...        ...   
14511   0.544776   0.507463   0.492537   0.537313   0.485075   0.455224   
14512   0.495652   0.573913   0.539130   0.547826   0.460870   0.495652   
14513   0.424084   0.549738   0.481675   0.492147   0.497382   0.523560   
14514   0.487179   0.500000   0.551282   0.589744   0.461538   0.576923   
14515   0.480000   0.600000   0.420000   0.420000   0.420000   0.440000   

       Feature_6  Feature_7  Feature_8  Feature_9  Feature_10  Feature_11  \
0       0.510460   0.493724   0.464435   0.456067    0.531381    0.451883   
1       0.477273   0.636364   0.409091   0.454545    0.409091    0.477273   
2       0.482412   0.477387   0.482412   0.522613    0.477387    0.467337   
3       0.465347   0.504950   0.495050   0.475248    0.495050    0.495050   
4       0.558559   0.522523   0.504505   0.495495    0.522523    0.450450   
...          ...        ...        ...        ...         ...         ...   
14511   0.470149   0.462687   0.597015   0.544776    0.514925    0.417910   
14512   0.573913   0.495652   0.556522   0.556522    0.573913    0.547826   
14513   0.476440   0.518325   0.507853   0.460733    0.507853    0.581152   
14514   0.487179   0.500000   0.525641   0.448718    0.589744    0.525641   
14515   0.500000   0.480000   0.560000   0.540000    0.540000    0.460000   

       Feature_12  Feature_13  Feature_14  Feature_15  Feature_16  Feature_17  \
0        0.502092    0.493724    0.510460    0.485356    0.472803    0.493724   
1        0.522727    0.454545    0.545455    0.477273    0.545455    0.545455   
2        0.517588    0.482412    0.492462    0.462312    0.507538    0.502513   
3        0.465347    0.475248    0.425743    0.445545    0.504950    0.485149   
4        0.504505    0.504505    0.522523    0.450450    0.576577    0.405405   
...           ...         ...         ...         ...         ...         ...   
14511    0.537313    0.514925    0.597015    0.462687    0.500000    0.500000   
14512    0.565217    0.556522    0.530435    0.530435    0.573913    0.486957   
14513    0.513089    0.523560    0.539267    0.502618    0.549738    0.492147   
14514    0.423077    0.551282    0.358974    0.500000    0.448718    0.474359   
14515    0.520000    0.440000    0.540000    0.520000    0.460000    0.540000   

       Feature_18  Feature_19  Feature_20  Feature_21  Feature_22  Feature_23  \
0        0.485356    0.502092    0.493724    0.485356    0.476987    0.510460   
1        0.522727    0.545455    0.500000    0.568182    0.431818    0.500000   
2        0.487437    0.522613    0.537688    0.417085    0.452261    0.532663   
3        0.435644    0.425743    0.495050    0.455446    0.485149    0.504950   
4        0.522523    0.432432    0.423423    0.567568    0.468468    0.468468   
...           ...         ...         ...         ...         ...         ...   
14511    0.522388    0.492537    0.514925    0.514925    0.492537    0.485075   
14512    0.547826    0.591304    0.547826    0.504348    0.556522    0.556522   
14513    0.523560    0.465969    0.492147    0.481675    0.534031    0.471204   
14514    0.564103    0.551282    0.512821    0.525641    0.564103    0.435897   
14515    0.520000    0.500000    0.680000    0.480000    0.420000    0.580000   

       Feature_24  Feature_25  Feature_26  Feature_27  Feature_28  Feature_29  \
0        0.489540    0.472803    0.464435    0.472803    0.464435    0.527197   
1        0.522727    0.500000    0.545455    0.431818    0.409091    0.45

### 2.  Functions to evaluate feature quality

In [6]:
def precision_at_k(df, column1, column2, k=10, largest1=True, largest2=False):
    """
    Compute Precision at k between two columns.
    
    Parameters:
    column1 (pd.Series): First column of data.
    column2 (pd.Series): Second column of data.
    k (int): Number of top elements to consider.
    
    Returns:
    float: Precision at k score.
    """
    # Get the top k indices for both columns
    if largest1:
        top_k_col1 = df[column1].nlargest(k).index
    else:
        top_k_col1 = df[column1].nsmallest(k).index
    
    if largest2:
        top_k_col2 = df[column2].nlargest(k).index
    else:
        top_k_col2 = df[column2].nsmallest(k).index
    
    # Compute the number of common indices
    common_indices = len(set(top_k_col1).intersection(set(top_k_col2)))
    
    # Precision at k
    precision = common_indices / k
    
    return precision


def find_best_random_combination(df, target_column, max_columns=3, num_samples=100):
    """
    Finds the best combination of at most `max_columns` columns that when added together with some set of weights
    results in the largest Spearman rank correlation with the target column by randomly sampling combinations.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    target_column (str): The name of the target column.
    max_columns (int): The maximum number of columns to combine.
    num_samples (int): The number of random combinations to sample.

    Returns:
    tuple: Best combination of columns, corresponding weights, and the highest Spearman rank correlation.
    """
    
    columns = [col for col in df.columns if col != target_column]
    best_combination = None
    best_weights = None
    highest_correlation = -1
    
    weight_values = [0.1, 0.5, 1., \
                     -0.1, -0.5, -1.]

    for _ in range(num_samples):
        # Randomly choose the number of columns to combine
        num_columns = random.randint(1, max_columns)
        combination = random.sample(columns, num_columns)
        
        comb_df = df[combination]
        
        for weights in product(weight_values, repeat=len(combination)):
            weights = np.array(weights)
            if np.sum(weights) == 0:
                continue

            weights /= np.sum(weights)  # Normalize weights
            
            combined = np.dot(comb_df, weights)
            df['combined'] = combined
            correlation = precision_at_k(df, 'combined', target_column)
            #correlation, _ = spearmanr(combined, df[target_column])

            if abs(correlation) > highest_correlation:
                highest_correlation = abs(correlation)
                best_combination = combination
                best_weights = weights
    
    return best_combination, best_weights, highest_correlation


In [7]:
def get_corr_df(results_df, ks, target_col, largest2=False):
    results_df = results_df.reset_index(drop=True)
    corr_df = pd.DataFrame()
    for column in results_df.columns:

        correlation, _ = spearmanr(results_df[column], results_df[target_col])

        tmp = {'column':column,
               'pearson_corr':results_df[column].corr(results_df[target_col]),
               'spearman_corr':correlation}
        tmp[f'prec_k_score'] = 0
        
        for k in ks:
            prec_k_largest = precision_at_k(results_df, column, target_col, k=k, largest1=True, largest2=largest2)
            prec_k_smallest = precision_at_k(results_df, column, target_col, k=k, largest1=False, largest2=largest2)
            prec_k = max(prec_k_largest, prec_k_smallest)
            tmp[f'prec_{k}'] = prec_k
            tmp[f'prec_k_score'] += prec_k

        corr_df = pd.concat([corr_df, pd.DataFrame([tmp])])
    corr_df = corr_df.reset_index(drop=True)
    return corr_df

### 3.  Looking at a Single Dataset

In [8]:
corr_df_error = get_corr_df(results_df, [1, 5, 10, 25], 'error', largest2=False)
corr_df_ood_acc = get_corr_df(results_df, [1, 5, 10, 25], 'ensemble_acc_val_ood', largest2=True)

In [9]:
corr_df_error.nlargest(10, 'prec_k_score')

,column,pearson_corr,spearman_corr,prec_k_score,prec_1,prec_5,prec_10,prec_25
1152,error,1.000000,1.000000,4.00,1.0,1.0,1.0,1.00
827,std_cos_val_ood_noise,0.478576,0.415149,0.48,0.0,0.2,0.2,0.08
120,ensemble_logisticR_train_positive_pred_percent,-0.313429,-0.363937,0.22,0.0,0.0,0.1,0.12
275,ensemble_logisticR_train_positive_pred_percent...,-0.313429,-0.363937,0.22,0.0,0.0,0.1,0.12
430,ensemble_logisticR_train_positive_pred_percent...,-0.313429,-0.363937,0.22,0.0,0.0,0.1,0.12
448,min_model_recall_val_ood,-0.018730,-0.060777,0.22,0.0,0.0,0.1,0.12
585,ensemble_logisticR_train_positive_pred_percent...,-0.313429,-0.363937,0.22,0.0,0.0,0.1,0.12
740,ensemble_logisticR_train_positive_pred_percent...,-0.313429,-0.363937,0.22,0.0,0.0,0.1,0.12
895,ensemble_logisticR_train_positive_pred_percent...,-0.313429,-0.363937,0.22,0.0,0.0,0.1,0.12
1052,ensemble_logisticR_train_positive_pred_percent...,-0.313429,-0.363937,0.22,0.0,0.0,0.1,0.12


In [10]:
corr_df_ood_acc.nlargest(10, 'prec_k_score')

,column,pearson_corr,spearman_corr,prec_k_score,prec_1,prec_5,prec_10,prec_25
404,ensemble_acc_val_ood,1.000000,1.000000,4.00,1.0,1.0,1.0,1.00
183,mutual_misclassification_mean,-0.704678,-0.705562,1.56,1.0,0.2,0.2,0.16
92,ensemble_auc,0.248433,0.257917,1.38,1.0,0.2,0.1,0.08
119,ensemble_logisticR_train_auc,0.218547,0.216201,1.38,1.0,0.2,0.1,0.08
274,ensemble_logisticR_train_auc_val_id,0.218547,0.216201,1.38,1.0,0.2,0.1,0.08
429,ensemble_logisticR_train_auc_val_ood,0.218547,0.216201,1.38,1.0,0.2,0.1,0.08
584,ensemble_logisticR_train_auc_val_id_noise,0.218547,0.216201,1.38,1.0,0.2,0.1,0.08
739,ensemble_logisticR_train_auc_val_ood_noise,0.218547,0.216201,1.38,1.0,0.2,0.1,0.08
894,ensemble_logisticR_train_auc_val_id_corrupt,0.218547,0.216201,1.38,1.0,0.2,0.1,0.08
1051,ensemble_logisticR_train_auc_val_ood_corrupt,0.218547,0.216201,1.38,1.0,0.2,0.1,0.08


### 4. Average over all datasets

In [10]:
dataset_results = {}
datasets = ['adult_tf', 'heloc_tf', 'compas', 'german']
save_path = '/proj/mcavoy_lab/data/evo_models/exps/cheating_fitness_0802_EOD/'

for i, dataset in enumerate(datasets):
    # load datasets
    if dataset not in dataset_results.keys():
        results_df = get_results_df(save_path + f'{dataset}')
        dataset_results[dataset] = results_df

        del results_df

(8447, 1241)
(10573, 1249)
(12550, 1236)
(14516, 1285)
min_std_mmd_val_id
min_mmd_val_id
min_std_mmd_val_id
min_mmd_val_id


In [11]:
target = 'ensemble_auc_val_ood'
maximize=True
corr_dfs = {}
for dataset in datasets:
    corr_dfs[dataset] = get_corr_df(dataset_results[dataset], [1, 5, 10, 25, 50], target, largest2=maximize)

In [12]:
corr_df = pd.DataFrame()
for df in corr_dfs.values():
    corr_df = pd.concat([corr_df, df])
corr_df = corr_df.groupby('column').mean().reset_index()

In [13]:
useful_rows = [x for x in corr_df['column'] if 'eature' not in x]
useful_rows = [x for x in useful_rows if 'ood' not in x]

In [18]:
corr_df[corr_df.column.isin(useful_rows)].nlargest(10, 'prec_25')

,column,pearson_corr,spearman_corr,prec_k_score,prec_1,prec_5,prec_10,prec_25,prec_50
92,ensemble_auc,0.560028,0.514471,0.595,0.0,0.05,0.175,0.18,0.19
196,ensemble_logisticR_train_precision,0.313645,0.292615,0.425,0.0,0.10,0.075,0.13,0.12
197,ensemble_logisticR_train_precision_val_id,0.313645,0.292615,0.425,0.0,0.10,0.075,0.13,0.12
198,ensemble_logisticR_train_precision_val_id_corrupt,0.313645,0.292615,0.425,0.0,0.10,0.075,0.13,0.12
200,ensemble_logisticR_train_precision_val_id_noise,0.313645,0.292615,0.425,0.0,0.10,0.075,0.13,0.12
163,ensemble_logisticR_train_acc,0.293033,0.278034,0.340,0.0,0.05,0.050,0.12,0.12
164,ensemble_logisticR_train_acc_val_id,0.293033,0.278034,0.340,0.0,0.05,0.050,0.12,0.12
165,ensemble_logisticR_train_acc_val_id_corrupt,0.293033,0.278034,0.340,0.0,0.05,0.050,0.12,0.12
167,ensemble_logisticR_train_acc_val_id_noise,0.293033,0.278034,0.340,0.0,0.05,0.050,0.12,0.12
350,ensemble_oob_auc,0.564720,0.503026,0.565,0.0,0.20,0.125,0.12,0.12
